In [1]:
import pandas as pd
import copy
import random
from load_dataset import load
from classifier import NeuralNetwork, LogisticRegression, SVM
from utils import *
from metrics import *  # include fairness and corresponding derivatives
from expl import explanation_candidate_generation, get_top_k_expl
from influence import *

random.seed(1)
np.random.seed(1)
torch.manual_seed(1)

In [2]:
# ignore all the warnings
import warnings
warnings.filterwarnings('ignore') 

**Parameters can be changed:**

**dataset**: german / sqf / adult

**clf_name**: LogisticRegression / SVM / NeuralNetwork

**metric**: The metric for debugging. 0 -> spd, 1 -> tpr parity (equal opportunity), 2 -> predictive parity

**support**: lower bound of pattern size

**support_small**: upper bound of pattern size

**duplicates**: make duplicates of datasets to test runtime vs dataset size with the same feature number and relationships

In [3]:
dataset = 'german'
clf_name = 'LogisticRegression'
metric = 0
support = 0.05
support_small = 0.3

**Load Dataset**

In [4]:
X_train, X_test, y_train, y_test = load(dataset)

In [5]:
duplicates = 1
make_duplicates = lambda x, d: pd.concat([x]*d, axis=0).reset_index(drop=True)
X_train = make_duplicates(X_train, duplicates)
X_test = make_duplicates(X_test, duplicates)
y_train = make_duplicates(y_train, duplicates)
y_test = make_duplicates(y_test, duplicates)

In [6]:
X_train_orig = copy.deepcopy(X_train)
X_test_orig = copy.deepcopy(X_test)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

**Model & Loss function**

In [7]:
clf = eval(clf_name)(input_size=X_train.shape[-1])

num_params = len(convert_grad_to_ndarray(list(clf.parameters())))
if isinstance(clf, LogisticRegression):
    loss_func = logistic_loss_torch
elif isinstance(clf, SVM):
    loss_func = svm_loss_torch
elif isinstance(clf, NeuralNetwork):
    loss_func = nn_loss_torch
else:
    raise NotImplementedError

**Metrics: Initial state**

In [8]:
clf = eval(clf_name)(input_size=X_train.shape[-1])

clf.fit(X_train, y_train)

y_pred_test = clf.predict_proba(X_test)
y_pred_train = clf.predict_proba(X_train)

spd_0 = computeFairness(y_pred_test, X_test_orig, y_test, 0, dataset)
print("Initial statistical parity: ", spd_0)

tpr_parity_0 = computeFairness(y_pred_test, X_test_orig, y_test, 1, dataset)
print("Initial TPR parity: ", tpr_parity_0)

predictive_parity_0 = computeFairness(y_pred_test, X_test_orig, y_test, 2, dataset)
print("Initial predictive parity: ", predictive_parity_0)

accuracy_0 = computeAccuracy(y_test, y_pred_test)
print("Initial accuracy: ", accuracy_0)

Initial statistical parity:  -0.09527580118738121
Initial TPR parity:  -0.07785149359511678
Initial predictive parity:  -0.10136869102808022
Initial accuracy:  0.755


**Select delta fairness function depending on selected metric**

In [9]:
metric_val = [spd_0, tpr_parity_0, predictive_parity_0][metric]
del_F_del_theta = get_del_F_del_theta(clf, X_test_orig, X_test, y_test, dataset, metric)

**Pre-compute: (1) Hessian $H_{\theta}$ (2) del_L_del_theta for each training data point**

In [10]:
hessian_all_points = get_hessian_all_points(clf, X_train, y_train, loss_func)

100%|██████████| 800/800 [00:09<00:00, 82.54it/s]


In [11]:
del_L_del_theta = get_del_L_del_theta(clf, X_train, y_train, loss_func)

**Hessian vector product: $H_{\theta}^{-1}v$**

In [12]:
hinv_v, hinv = get_hinv_v(hessian_all_points, del_F_del_theta)

**Removal-based explanation generation**

In [13]:
# influence lower bound for the first-level patterns.
# The first-level patterns with influencelower than this value would be filtered out
del_f_threshold = 0.1 * metric_val

In [14]:
%%time
containment_df = explanation_candidate_generation(X_train_orig, X_train, y_train, dataset, del_L_del_theta,
                                                  hessian_all_points, hinv, del_F_del_theta,
                                                  del_f_threshold=del_f_threshold, support=support,
                                                  support_small=support_small)

Generated:  62  1-candidates
Generated:  341  2-candidates
Generated:  2
Generated: 2429   3 -candidates
Generated:  3
Generated: 10798   4 -candidates
# candidates left:  12861
CPU times: user 14.7 s, sys: 77.1 ms, total: 14.8 s
Wall time: 14.8 s


**Containment-based filtering**

In [15]:
topk = get_top_k_expl(containment_df, X_train_orig, containment_threshold=0.2, k=5)
print(topk.top_explanations)  # in the format of [(pattern, indices)]

{'["age=1", "gender=0"]': (Int64Index([ 13,  37,  76, 196, 212, 233, 241, 266, 285, 294, 333, 342, 354,
            357, 370, 376, 398, 430, 435, 444, 494, 526, 537, 542, 545, 553,
            562, 567, 588, 592, 598, 608, 613, 656, 671, 701, 727, 737, 749,
            797],
           dtype='int64'), 0.9832245805874269), '["age=1", "credit_hist=0", "gender=1"]': (Int64Index([ 14,  17,  62,  63,  64,  80,  86,  89,  97, 124, 133, 139, 162,
            164, 168, 182, 208, 214, 254, 275, 301, 303, 306, 329, 337, 344,
            369, 399, 409, 411, 462, 468, 484, 500, 523, 556, 578, 585, 614,
            623, 644, 668, 673, 679, 707, 713, 719, 742, 793, 796],
           dtype='int64'), 0.6207703170367591), '["credit_amt=0", "install_rate=4", "install_plans=0", "status=0"]': (Int64Index([ 14,  17,  43,  72,  82, 102, 112, 120, 124, 166, 170, 223, 224,
            241, 245, 254, 281, 301, 317, 328, 351, 355, 357, 365, 371, 406,
            425, 430, 434, 482, 483, 492, 501, 511, 535, 550, 

**Integrate the patterns and other info into more comprehensive results**

In [16]:
explanations = list(topk.top_explanations.keys())
idxs = [v[1] for v in topk.top_explanations.values()]
supports = list()
scores = list()
gt_scores = list()
infs = list()
gts = list()
new_accs = list()
for e in explanations:
    idx = get_subset(json.loads(e), X_train_orig)
    X = np.delete(X_train, idx, 0)
    y = y_train.drop(index=idx, inplace=False)
    clf.fit(np.array(X), np.array(y))
    y_pred = clf.predict_proba(np.array(X_test))
    new_acc = computeAccuracy(y_test, y_pred)
    inf_gt = computeFairness(y_pred, X_test_orig, y_test, 0, dataset) - metric_val
    
    condition = containment_df.predicates.apply(lambda x: x==json.loads(e))
    supports.append(float(containment_df[condition]['support']))
    scores.append(float(containment_df[condition]['score']))
    infs.append(float(containment_df[condition]['2nd-inf']))
    gts.append(inf_gt/(-metric_val))
    gt_scores.append(inf_gt*100/float(containment_df[condition]['support']))
    new_accs.append(new_acc)


expl = [explanations, supports, scores, gt_scores, infs, gts, new_accs]
expl = np.array(expl).T.tolist()

explanations = pd.DataFrame(expl, columns=["explanations", "support", "score", "gt-score", "2nd-inf(%)", "gt-inf(%)", "new-acc"])
explanations['score'] = explanations['score'].astype(float)
explanations['gt-score'] = explanations['gt-score'].astype(float)
explanations['support'] = explanations['support'].astype(float)
explanations['2nd-inf(%)'] = explanations['2nd-inf(%)'].astype(float)/(-metric_val)
explanations['gt-inf(%)'] = explanations['gt-inf(%)'].astype(float)
explanations['new-acc'] = explanations['new-acc'].astype(float)

pd.set_option('max_colwidth', 100)
explanations.sort_values(by=['score'], ascending=False)


,explanations,support,score,gt-score,2nd-inf(%),gt-inf(%),new-acc
0,"[""age=1"", ""gender=0""]",5.000,0.983225,1.079512,0.515989,0.566519,0.745
1,"[""age=1"", ""credit_hist=0"", ""gender=1""]",6.250,0.620770,0.660626,0.407219,0.433364,0.755
2,"[""credit_amt=0"", ""install_rate=4"", ""install_plans=0"", ""status=0""]",6.500,0.418993,0.413260,0.285849,0.281938,0.755
3,"[""duration=1"", ""employment=4"", ""num_liable=1"", ""residence=4""]",5.375,0.404900,0.403639,0.228425,0.227714,0.740
4,"[""credit_hist=2"", ""gender=0"", ""housing_A152=1"", ""savings=0""]",8.000,0.393087,0.385020,0.330062,0.323289,0.760
